In [47]:
import pandas as pd
melb_df = pd.read_csv('data/melb_data.csv', sep=',')#,index_col=0)
# не читить строки skiprows= skiprows=1
# читить столбцы usecols=
# melb_df.head() 
# melb_df.shape
# melb_df.info()
# melb_df.columns
# melb_df['Date'].value_counts()

подготовка

In [48]:
# Удаление строк и столбцов в таблице отвечает метод drop()
melb_df.drop(['index','Coordinates'],axis=1,inplace=True,    errors='ignore')
# Перевели в дату
melb_df['Date']=pd.to_datetime(melb_df['Date'],dayfirst=True,errors='ignore')
# изменили тип переменных
melb_df['Car'] = melb_df['Car'].astype('int64')
melb_df['Bedroom'] = melb_df['Bedroom'].astype('int64')
melb_df['Bathroom'] = melb_df['Bathroom'].astype('int64')
melb_df['Propertycount'] = melb_df['Propertycount'].astype('int64')
melb_df['YearBuilt'] = melb_df['YearBuilt'].astype('int64')
# сумма комнат+спален+ванных комнат
total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
# средняя площадь конаты       полщадь здания / сумма комнат+спален+ванных комнат
melb_df['MeanRoomsSquare']= melb_df['BuildingArea'] / total_rooms
# соотношения площади здания к площади участка
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
# номер месяц продажи
melb_df['MonthSale'] = melb_df['Date'].dt.month
# номер дня недели продажи
melb_df['WeekdaySale'] = melb_df['Date'].dt.weekday+1
#«признак-мигалку» выходного дня
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(lambda x: 1 if x>5 else 0)
# заменили признак года постройки здания
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
melb_df.drop('YearBuilt', axis=1 ,inplace=True, errors='ignore')


In [49]:
# признак подтипа улицы

# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

street_types = melb_df['Address'].apply(get_street_type)

exclude_dict = {'Avenue':'Av', 'Boulevard':'Bvd', 'Parade':'Pde',
                    'Crescent':'Ct','Strand':'St','Grove':'Gr'}

street_types = street_types.apply(lambda x:exclude_dict[x] if x in exclude_dict else x)

popular_stypes =street_types.value_counts().nlargest(10).index

melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
melb_df = melb_df.drop('Address', axis=1, errors='ignore')

# уменьшили число уникальных наименований агентств по недвижимости
popular_stypes =melb_df['SellerG'].value_counts().nlargest(49).index
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_stypes else 'other')

# уменьшили число уникальных наименований число пригородов
popular_stypes =melb_df['Suburb'].value_counts().nlargest(119).index
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in popular_stypes else 'other')

# выделили категориальные признаки и преобразовали их в тип данных
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
 
# заменили сокращённые названия категорий признака типа объекта       
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})

# melb_df['Suburb'].value_counts()
# melb_df.nunique()        
# melb_df.head(1)
# melb_df.info()

.count()	Количество непустых значений
.mean()	Среднее значение
.min()	Минимальное значение
.max()	Максимальное значение
.var()	Дисперсия
.std()	Стандартное отклонение
.sum()	Сумма
.quantile(x)	Квантиль уровня x
.nunique()	Число уникальных значений
.moda()  Самое частое значение
.median() Медиана

index — номер строки
Suburb — наименование пригорода
Address — адрес
Rooms — количество комнат в помещении
Type — тип здания (h — дом, коттедж, вилла, терраса; u — блочный, дуплексный дом; t — таунхаус)
Price — цена помещения
Method — метод продажи 
SellerG — риэлторская компания
Date — дата продажи (в формате день/месяц/год)
Distance — расстояния до объекта от центра Мельбурна 
Postcode — почтовый индекс
Bedroom — количество спален
Bathroom — количество ванных комнат
Car — количество парковочных мест
Landsize — площадь прилегающей территории
BuildingArea — площадь здания
YearBuilt — год постройки
CouncilArea — региональное управление
Lattitude — географическая широта
Longitude — географическая долгота
Regionname — наименование района Мельбурна
Propertycount — количество объектов недвижимости в районе, выставленных на продажу
Coordinates — широта и долгота, объединённые в кортеж

In [51]:
# display(melb_df.info())
# melb_df.describe()
melb_df.head()
melb_df['MonthSale'].value_counts()

MonthSale
5     2029
7     1982
9     1845
6     1830
8     1550
11    1114
4      949
3      678
12     607
10     551
2      443
1        2
Name: count, dtype: int64